In [1]:
import pandas as pd
from crr import GraphService

In [ ]:
df = pd.read_csv("./response_1746819263900.csv)
gg = GraphService()
gg.create_graph(dataframe=df, limit_nodes=20)

2025-05-09 16:02:34.205 | INFO     | crr:__init__:26 - Initialized Graph service.
2025-05-09 16:02:34.205 | INFO     | crr:create_graph:111 - Creating graph from dataframe.
2025-05-09 16:02:34.206 | INFO     | crr:create_graph:116 - Preprocessing to identify target and batedor plates.
2025-05-09 16:02:34.207 | INFO     | crr:create_graph:120 - Iterating over dataframe rows to add nodes and edges.
2025-05-09 16:02:34.315 | INFO     | crr:create_graph:131 - Removing isolated nodes.
2025-05-09 16:02:34.316 | INFO     | crr:create_graph:135 - Limiting nodes in graph.
2025-05-09 16:02:34.321 | INFO     | crr:create_graph:140 - Graph created.


In [ ]:
net = gg.to_html()

net.show("grafo.html")

grafo.html


In [4]:
gg.to_png_improved()

2025-05-09 16:02:34.354 | INFO     | crr:to_png_improved:238 - Convertendo grafo para PNG com layout otimizado: grafo.png em ./
2025-05-09 16:02:34.360 | DEBUG    | crr:to_png_improved:257 - Diretório . verificado/criado.
2025-05-09 16:02:34.376 | DEBUG    | crr:to_png_improved:264 - Figura matplotlib criada com tamanho (20, 16).
2025-05-09 16:02:34.376 | INFO     | crr:to_png_improved:267 - Aplicando layout de força direcionada otimizado ao grafo.
2025-05-09 16:02:34.433 | INFO     | crr:to_png_improved:283 - Layout de força direcionada otimizado calculado.
2025-05-09 16:02:34.434 | INFO     | crr:to_png_improved:290 - Determinando estilos dos nós (cor, tamanho, forma, borda).
2025-05-09 16:02:34.434 | DEBUG    | crr:to_png_improved:329 - Estilos dos nós determinados.
2025-05-09 16:02:34.434 | INFO     | crr:to_png_improved:332 - Normalizando larguras das arestas com base no peso.
2025-05-09 16:02:34.435 | DEBUG    | crr:to_png_improved:386 - Pesos das arestas escalonados de [5.0, 9.0

PosixPath('grafo.png')